In [2]:
import nflgame
import numpy as np

Updating (2019, 'REG', 6)


In [3]:
# Surrender Index Formula:
# Field Position * First Down Distance * Score of Game * Clock

# Surrender Index Calculations:
# Field Position: At or inside 40, base score = 1 + 10% per yard past 40 to 50, + 20% per yard past 50
# First Down Distance: 4th and 1 = no discount, 4th and 2-3 = 20% off, 4th and 4-6 = 40% off, 4th and 7-9 = 60% off, 4th and 10+ = 80% off
# Score of Game: Tied = 2x multiplier, Losing by 2+ score = 3x, Losing by 1 score = 4x, winning = 1x
# Clock: Applies if losing/tied after halftime = ((x*0.001)^3) + 1 for each passing second after halftime

In [4]:
years = range(2009, 2019)
weeks = range(1, 18)
seasons_dict = {}

for i in years:
    week_dict = {}
    for j in weeks:
        try:
            week_dict[j] = nflgame.games(i, week=j)
        except:
            print("YEAR {} WEEK {} FAILED".format(i, j))
    
    seasons_dict[i] = week_dict

# seasons_dict now contains all games from all weeks that didn't fail

YEAR 2017 WEEK 1 FAILED
YEAR 2018 WEEK 16 FAILED


In [5]:
# Iterate through seasons_dict and find each punt.
punts = []

for i in years:
    for j in weeks:
        try:
            for game in seasons_dict[i][j]:
                for play in nflgame.combine_plays([game]):
                    if("punts" in play.desc):
                        punts.append(play)
        except:
            print("Exception Found in SEASON {} WEEK {}".format(i, j))

# These punts are going to be run through the surrender index.
len(punts)

Exception Found in SEASON 2017 WEEK 1
Exception Found in SEASON 2018 WEEK 16


24185

In [6]:
def get_field_position(play):
    """
    Gets the field position of the play.
    
    Arg: play is the play being analyzed.
    Returns: the field position as an int from 1-100. 
    1-50 represents own half, 50-100 represents opponent half.
    """
    
    if(play.data['yrdln'] == '50'):
        return 50
    
    team_side = play.data['yrdln'].split()[0]
    yard_line = int(play.data['yrdln'].split()[1])
    pos_team = play.data['posteam']
    
    if(pos_team != team_side):
        return (50 + (50 - yard_line))
    
    else:
        return yard_line

In [7]:
def get_first_down_distance(play):
    """
    Gets the first down distance required of the play.
    Arg: play: a Play object
    Returns: First down distance as an int.
    """
    return play.data['ydstogo']

In [8]:
def score_differential(play):
    """
    Gets the score differential for the team in possesion at
    the time of the play.
    Arg: play: a Play object
    Returns: Score differential as an int (from perspective of possession team)
    """
    
    # Identify home team and away team
    home_team = play.drive.game.home
    away_team = play.drive.game.away
    
    # Identify team in possession
    pos_team = play.data['posteam']
    
    # Score is the score list. score[0] is away, score[1] is home
    score = score_at_play(play)
    
    # Calculate score difference from perspective of possession team
    if(pos_team == home_team):
        diff = score[1] - score[0]
    else:
        diff = score[0] - score[1]
        
    return diff

In [9]:
def clock_multiplier(play):
    """
    Gets multiplier to apply to surrender index at the time of the play.
    Arg: play: Play object.
    Returns: float multiplier.
    """
    # Clock: Applies if losing/tied after halftime = ((x*0.001)^3) + 1 for each passing second after halftime
    # Need to account for overtime as well.
    if(play.data['qtr'] == 3):
        # past halftime, 3rd quarter
        time = seconds(play) # Seconds counts how many seconds are in the clock
        
    elif(play.data['qtr'] == 4):
        # past halftime, 4th quarter
        # Calculations must add +15 min worth of seconds to this total
        time = seconds(play) + (15 * 60)
        
    else:
        return 1
        
    multiplier = ((time * 0.001) ** 3) + 1
    return multiplier

In [10]:
def seconds(play):
    """
    Counts how many seconds are left on the clock at the time of a play.
    Arg: play: Play object.
    Returns: int of number of seconds.
    """
    # Gather time string list
    time_str = play.data['time'].split(":")
    time_str[0] = int(time_str[0])
    time_str[1] = int(time_str[1])
    
    time = (60 * time_str[0]) + time_str[1]
    return time

In [11]:
def yard_line_multiplier(play):
    """
    Converts yard line in the form of 1-100 
    Field Position: At or inside 40, base score = 1 + 10% per yard past 40 to 50, + 20% per yard past 50
    Arg: play: Play object
    Returns: float to be used as multiplier
    """
    field_pos = get_field_position(play)
    multiplier = 1
    
    if(field_pos <= 40):
        return 1
    
    elif(field_pos >= 41 and field_pos <= 50):
        numloops = field_pos - 40
        for i in range(numloops):
            multiplier = multiplier * 1.1
        
    elif(field_pos > 50):
        numloops = field_pos - 50
        for i in range(10):
            multiplier = multiplier * 1.1
        for j in range(numloops):
            multiplier = multiplier * 1.2

    return multiplier

In [12]:
def first_down_distance_multiplier(play):
    """
    Converts the first down distance to the appropriate multiplier
    First Down Distance: 4th and 1 = no discount, 4th and 2-3 = 20% off, 4th and 4-6 = 40% off,
                         4th and 7-9 = 60% off, 4th and 10+ = 80% off
    Arg: play: Play object
    Returns: float to be used as multiplier
    """
    multiplier = get_first_down_distance(play)
    
    if(multiplier <= 1):
        return 1
    elif(multiplier == 2 or multiplier == 3):
        return 0.8
    elif(multiplier >= 4 and multiplier <= 6):
        return 0.6
    elif(multiplier >= 7 and multiplier <= 9):
        return 0.4
    elif(multiplier >= 10):
        return 0.2

In [13]:
def score_differential_multiplier(play):
    """
    Applies a multiplier based on the difference between scores.
    When tied - 2x.
    When losing by 2+ scores - 3x.
    When losing by 1 score - 4x.
    When winning - 1x.
    Arg: diff: integer representing score differential.
    Returns: integer to be used as multiplier.
    """
    diff = score_differential(play)
    
    if(diff > 0):
        return 1
    elif(diff == 0):
        return 2
    elif(diff < -8):
        return 3
    else:
        return 4

In [14]:
def surrender_index(play):
    """
    Calculates the surrender index of the punt.
    Arg: play: Play object.
    Returns: float of surrender index score.
    """
    surr = yard_line_multiplier(play) * first_down_distance_multiplier(play) * score_differential_multiplier(play) * clock_multiplier(play)
    return surr

In [15]:
def surrender_index_all_punts(punt_list):
    """
    Calculates surrender index of all punts in the list.
    Arg: play_list: List of Play objects.
    Returns: List of float surrender indexes for all punts
    """
    new_list = []
    for i in punt_list:
        surr = surrender_index(i)
        new_list.append(surrender_index(i))
        
    return new_list

In [16]:
# Safety: Desc will have SAFETY in the play. Posteam = team w/ball. Other team gets 2 points.
# Field goal: "field goal is No Good" or "field goal is GOOD"
# Touchdown: Desc will have TOUCHDOWN. Posteam = team w/ball gets 6 pts.
# Extra point: "extra point is No Good" or "extra point is GOOD"
# Two-point conversion: "TWO-POINT CONVERSION ATTEMPT." ... "ATTEMPT SUCCEEDS" or "ATTEMPT FAILS"

# Turnover Cases:
# 1. Interception - handle by awarding points to other team
# 2. Fumble - Check who it's recovered by, see if it's the other team. If so, award to other team.
# For fumbles, RECOVERED by instead of recovered by indicates turnover.
# 3. Multiple fumbles: Good luck kid.

In [17]:
def score_at_play(play):
    """
    Gets the current score at the time of the play.
    (Does so by iterating up until the point of the play.)
    Arg: play: a Play object
    Returns: List containing score of home team, score of away team
    """
    # Identify home team and away team
    home_team = play.drive.game.home
    away_team = play.drive.game.away
    
    # Initialize score at beginning of game; score[0] = away, score[1] = home
    score = [0, 0]
    
    # Initialize current game's plays
    current_game = play.drive.game
    current_game_plays = nflgame.combine_plays([current_game])
    
    for i in current_game_plays:
        # Using this to truncate in case of play reviews
        description = i.desc
        
        # If this is the same play as our identified play
        if(play.desc == i.desc):
            return score
        
        # If the play was reviewed and reversed, truncate to actual result.
        if("and the play was REVERSED" in description):
            description = description.split("and the play was REVERSED.")[1]
        
        # Identify team in possession
        pos_team = i.data['posteam']
        
        # Identify if interception happens
        if("INTERCEPTED" in description):
            if(home_team == pos_team):
                pos_team = away_team # Possession changes hands
            else:
                pos_team = home_team
        
        # Identify if fumble recovery turnover happens
        if("RECOVERED by" in description):
            if("MUFFS" not in description): # Muffed punt recovered by kicking team, everything stays the same
                if(home_team == pos_team):
                    pos_team = away_team # Possession changes hands
                else:
                    pos_team = home_team
        
        # If a touchdown was scored
        if("TOUCHDOWN" in description):
            if(home_team == pos_team):
                score[1] += 6
            else:
                score[0] += 6
        
        # If a field goal is scored
        elif("field goal is GOOD" in description):
            if(home_team == pos_team):
                score[1] += 3
            else:
                score[0] += 3
        
        # If an extra point is scored
        elif("extra point is GOOD" in description):
            if(home_team == pos_team):
                score[1] += 1
            else:
                score[0] += 1
                
        # If two-point conversion is scored
        elif("TWO-POINT CONVERSION ATTEMPT" in description and "ATTEMPT SUCCEEDS" in description):
            if(home_team == pos_team):
                score[1] += 2
            else:
                score[0] += 2
                
        # If safety is scored
        elif("SAFETY" in description):
            if(home_team == pos_team):
                score[0] += 2 # Because on a safety, the other team gets points
            else:
                score[1] += 2
    
    # Ideally we never have to return from here because we find the play
    print("PLAY NOT FOUND")
    return score

In [25]:
def get_final_stackrank(punts):
    """
    Gets the final stackrank for all punts in the list (punts).
    Arg: punts: a List of Play objects.
    Returns: stackranked array of punts and their surrender indices.
    """
    # Find surrender index for each punt
    all_surrender_index = surrender_index_all_punts(punts)
    
    # Grab final numpy array sorted to find punts with the highest surrender index
    final = []

    for i in range(len(all_surrender_index)):
        final.append((punts[i], all_surrender_index[i]))

    final = np.array(final)

    final[:, 1]
    final = final[final[:, 1].argsort()]
    return final[::-1]

In [27]:
get_final_stackrank(punts)

[[<nflgame.game.Play object at 0x00000236A3342588> 484.48693645995246]
 [<nflgame.game.Play object at 0x000002368ACEC518> 410.3698054663298]
 [<nflgame.game.Play object at 0x00000236C817EE48> 380.6481374558097]
 ...
 [<nflgame.game.Play object at 0x00000236CCF7ED68> 0.2]
 [<nflgame.game.Play object at 0x00000237178DCDD8> 0.2]
 [<nflgame.game.Play object at 0x00000236995FD7B8> 0.2]]


In [232]:
# Debug final score discrepancies
for i in years:
    for j in weeks:
        weeks_games = nflgame.games(i, week=j)
        for game in weeks_games:
            game_plays = nflgame.combine_plays([game])
            for play in game_plays:
                last_play_iter = play
            print("SEASON {} WEEK {}".format(i, j), score_at_play(last_play_iter))
            print(last_play_iter.drive.game.nice_score())

SEASON 2009 WEEK 1 [10, 10]
TEN (10) at PIT (13)
SEASON 2009 WEEK 1 [13, 19]
MIA (7) at ATL (19)
SEASON 2009 WEEK 1 [23, 38]
KC (24) at BAL (38)
SEASON 2009 WEEK 1 [30, 16]
PHI (38) at CAR (10)
SEASON 2009 WEEK 1 [12, 7]
DEN (12) at CIN (7)
SEASON 2009 WEEK 1 [40, 19]
MIN (34) at CLE (20)
SEASON 2009 WEEK 1 [24, 6]
NYJ (24) at HOU (7)
SEASON 2009 WEEK 1 [12, 14]
JAC (12) at IND (14)
SEASON 2009 WEEK 1 [26, 51]
DET (27) at NO (45)
SEASON 2009 WEEK 1 [34, 21]
DAL (34) at TB (21)
SEASON 2009 WEEK 1 [20, 16]
SF (20) at ARI (16)
SEASON 2009 WEEK 1 [17, 22]
WAS (17) at NYG (23)
SEASON 2009 WEEK 1 [6, 28]
STL (0) at SEA (28)
SEASON 2009 WEEK 1 [15, 21]
CHI (15) at GB (21)
SEASON 2009 WEEK 1 [23, 25]
BUF (24) at NE (25)
SEASON 2009 WEEK 1 [24, 20]
SD (24) at OAK (20)
SEASON 2009 WEEK 2 [20, 28]
CAR (20) at ATL (28)
SEASON 2009 WEEK 2 [28, 13]
MIN (27) at DET (13)
SEASON 2009 WEEK 2 [31, 23]
CIN (31) at GB (24)
SEASON 2009 WEEK 2 [30, 17]
ARI (31) at JAC (17)
SEASON 2009 WEEK 2 [13, 10]
OAK (13

SEASON 2009 WEEK 13 [19, 13]
NYJ (19) at BUF (13)
SEASON 2009 WEEK 13 [39, 6]
PHI (34) at ATL (7)
SEASON 2009 WEEK 13 [6, 16]
TB (6) at CAR (16)
SEASON 2009 WEEK 13 [9, 17]
STL (9) at CHI (17)
SEASON 2009 WEEK 13 [13, 25]
DET (13) at CIN (23)
SEASON 2009 WEEK 13 [17, 27]
TEN (17) at IND (27)
SEASON 2009 WEEK 13 [18, 23]
HOU (18) at JAC (23)
SEASON 2009 WEEK 13 [43, 13]
DEN (44) at KC (13)
SEASON 2009 WEEK 13 [27, 24]
OAK (27) at PIT (24)
SEASON 2009 WEEK 13 [29, 30]
NO (33) at WAS (30)
SEASON 2009 WEEK 13 [30, 23]
SD (30) at CLE (23)
SEASON 2009 WEEK 13 [17, 30]
MIN (17) at ARI (30)
SEASON 2009 WEEK 13 [30, 24]
DAL (24) at NYG (31)
SEASON 2009 WEEK 13 [17, 17]
SF (17) at SEA (20)
SEASON 2009 WEEK 13 [21, 22]
NE (21) at MIA (22)
SEASON 2009 WEEK 13 [14, 27]
BAL (14) at GB (27)
SEASON 2009 WEEK 14 [6, 13]
PIT (6) at CLE (13)
SEASON 2009 WEEK 14 [26, 23]
NO (26) at ATL (23)
SEASON 2009 WEEK 14 [3, 48]
DET (3) at BAL (48)
SEASON 2009 WEEK 14 [21, 14]
GB (21) at CHI (14)
SEASON 2009 WEEK 14

SEASON 2010 WEEK 7 [28, 17]
CLE (30) at NO (17)
SEASON 2010 WEEK 7 [16, 13]
WAS (17) at CHI (14)
SEASON 2010 WEEK 7 [19, 23]
SF (20) at CAR (23)
SEASON 2010 WEEK 7 [20, 18]
STL (17) at TB (18)
SEASON 2010 WEEK 7 [19, 30]
PHI (19) at TEN (37)
SEASON 2010 WEEK 7 [10, 31]
ARI (10) at SEA (22)
SEASON 2010 WEEK 7 [23, 20]
NE (23) at SD (20)
SEASON 2010 WEEK 7 [70, 14]
OAK (59) at DEN (14)
SEASON 2010 WEEK 7 [24, 27]
MIN (24) at GB (28)
SEASON 2010 WEEK 7 [50, 28]
NYG (41) at DAL (35)
SEASON 2010 WEEK 8 [22, 30]
DEN (16) at SF (24)
SEASON 2010 WEEK 8 [35, 17]
JAC (35) at DAL (17)
SEASON 2010 WEEK 8 [22, 14]
MIA (22) at CIN (14)
SEASON 2010 WEEK 8 [10, 10]
BUF (10) at KC (13)
SEASON 2010 WEEK 8 [19, 49]
WAS (25) at DET (37)
SEASON 2010 WEEK 8 [10, 23]
CAR (10) at STL (20)
SEASON 2010 WEEK 8 [9, 0]
GB (9) at NYJ (0)
SEASON 2010 WEEK 8 [31, 33]
TEN (25) at SD (33)
SEASON 2010 WEEK 8 [18, 28]
MIN (18) at NE (28)
SEASON 2010 WEEK 8 [36, 34]
TB (38) at ARI (35)
SEASON 2010 WEEK 8 [3, 33]
SEA (3) a

SEASON 2011 WEEK 2 [3, 48]
KC (3) at DET (48)
SEASON 2011 WEEK 2 [35, 38]
OAK (35) at BUF (38)
SEASON 2011 WEEK 2 [13, 26]
BAL (13) at TEN (26)
SEASON 2011 WEEK 2 [3, 32]
JAC (3) at NYJ (32)
SEASON 2011 WEEK 2 [27, 19]
CLE (27) at IND (19)
SEASON 2011 WEEK 2 [21, 22]
ARI (21) at WAS (22)
SEASON 2011 WEEK 2 [30, 20]
TB (24) at MIN (20)
SEASON 2011 WEEK 2 [30, 29]
GB (30) at CAR (23)
SEASON 2011 WEEK 2 [13, 30]
CHI (13) at NO (30)
SEASON 2011 WEEK 2 [0, 24]
SEA (0) at PIT (24)
SEASON 2011 WEEK 2 [24, 24]
DAL (27) at SF (24)
SEASON 2011 WEEK 2 [21, 35]
SD (21) at NE (35)
SEASON 2011 WEEK 2 [22, 24]
CIN (22) at DEN (24)
SEASON 2011 WEEK 2 [23, 13]
HOU (23) at MIA (13)
SEASON 2011 WEEK 2 [31, 35]
PHI (31) at ATL (35)
SEASON 2011 WEEK 2 [16, 27]
STL (16) at NYG (28)
SEASON 2011 WEEK 3 [10, 16]
JAC (10) at CAR (16)
SEASON 2011 WEEK 3 [33, 43]
HOU (33) at NO (40)
SEASON 2011 WEEK 3 [31, 30]
NE (31) at BUF (34)
SEASON 2011 WEEK 3 [16, 17]
MIA (16) at CLE (17)
SEASON 2011 WEEK 3 [14, 17]
DEN (14

SEASON 2011 WEEK 13 [24, 31]
IND (24) at NE (31)
SEASON 2011 WEEK 13 [38, 14]
SD (38) at JAC (14)
SEASON 2011 WEEK 14 [3, 14]
CLE (3) at PIT (14)
SEASON 2011 WEEK 14 [33, 33]
NE (34) at WAS (27)
SEASON 2011 WEEK 14 [16, 15]
OAK (16) at GB (46)
SEASON 2011 WEEK 14 [10, 37]
KC (10) at NYJ (37)
SEASON 2011 WEEK 14 [20, 19]
HOU (20) at CIN (19)
SEASON 2011 WEEK 14 [9, 24]
IND (10) at BAL (24)
SEASON 2011 WEEK 14 [28, 32]
MIN (28) at DET (34)
SEASON 2011 WEEK 14 [31, 23]
ATL (31) at CAR (23)
SEASON 2011 WEEK 14 [26, 10]
PHI (26) at MIA (10)
SEASON 2011 WEEK 14 [20, 33]
TB (14) at JAC (41)
SEASON 2011 WEEK 14 [22, 23]
NO (22) at TEN (17)
SEASON 2011 WEEK 14 [19, 21]
SF (19) at ARI (21)
SEASON 2011 WEEK 14 [10, 10]
CHI (10) at DEN (13)
SEASON 2011 WEEK 14 [9, 42]
BUF (10) at SD (37)
SEASON 2011 WEEK 14 [37, 34]
NYG (37) at DAL (34)
SEASON 2011 WEEK 14 [13, 29]
STL (13) at SEA (30)
SEASON 2011 WEEK 15 [13, 40]
JAC (14) at ATL (41)
SEASON 2011 WEEK 15 [31, 14]
DAL (31) at TB (15)
SEASON 2011 WE

...

SEASON 2014 WEEK 8 [43, 23]
BUF (43) at NYJ (23)
SEASON 2014 WEEK 8 [13, 13]
MIN (19) at TB (13)
SEASON 2014 WEEK 8 [30, 16]
HOU (30) at TEN (16)
SEASON 2014 WEEK 8 [20, 24]
PHI (20) at ARI (24)
SEASON 2014 WEEK 8 [13, 26]
OAK (13) at CLE (23)
SEASON 2014 WEEK 8 [34, 53]
IND (34) at PIT (51)
SEASON 2014 WEEK 8 [23, 44]
GB (23) at NO (44)
SEASON 2014 WEEK 8 [20, 17]
WAS (20) at DAL (17)
SEASON 2014 WEEK 9 [28, 10]
NO (28) at CAR (10)
SEASON 2014 WEEK 9 [23, 39]
JAC (23) at CIN (33)
SEASON 2014 WEEK 9 [17, 22]
TB (17) at CLE (22)
SEASON 2014 WEEK 9 [28, 16]
ARI (28) at DAL (17)
SEASON 2014 WEEK 9 [31, 20]
PHI (31) at HOU (21)
SEASON 2014 WEEK 9 [10, 24]
NYJ (10) at KC (24)
SEASON 2014 WEEK 9 [0, 37]
SD (0) at MIA (37)
SEASON 2014 WEEK 9 [26, 29]
WAS (26) at MIN (29)
SEASON 2014 WEEK 9 [13, 10]
STL (13) at SF (10)
SEASON 2014 WEEK 9 [27, 36]
DEN (21) at NE (43)
SEASON 2014 WEEK 9 [23, 29]
OAK (24) at SEA (30)
SEASON 2014 WEEK 9 [17, 49]
BAL (23) at PIT (43)
SEASON 2014 WEEK 9 [40, 24]
IND

SEASON 2015 WEEK 2 [20, 23]
MIA (20) at JAC (23)
SEASON 2015 WEEK 2 [19, 16]
DAL (20) at PHI (10)
SEASON 2015 WEEK 2 [17, 27]
SEA (17) at GB (27)
SEASON 2015 WEEK 2 [20, 7]
NYJ (20) at IND (7)
SEASON 2015 WEEK 3 [15, 38]
WAS (21) at NYG (32)
SEASON 2015 WEEK 3 [39, 28]
ATL (39) at DAL (28)
SEASON 2015 WEEK 3 [16, 39]
NO (22) at CAR (27)
SEASON 2015 WEEK 3 [17, 23]
PHI (24) at NYJ (17)
SEASON 2015 WEEK 3 [9, 19]
TB (9) at HOU (19)
SEASON 2015 WEEK 3 [14, 33]
SD (14) at MIN (31)
SEASON 2015 WEEK 3 [12, 6]
PIT (12) at STL (6)
SEASON 2015 WEEK 3 [17, 51]
JAC (17) at NE (51)
SEASON 2015 WEEK 3 [28, 23]
CIN (28) at BAL (24)
SEASON 2015 WEEK 3 [27, 20]
OAK (27) at CLE (20)
SEASON 2015 WEEK 3 [34, 39]
IND (35) at TEN (33)
SEASON 2015 WEEK 3 [7, 45]
SF (7) at ARI (47)
SEASON 2015 WEEK 3 [6, 20]
CHI (0) at SEA (26)
SEASON 2015 WEEK 3 [40, 14]
BUF (41) at MIA (14)
SEASON 2015 WEEK 3 [24, 12]
DEN (24) at DET (12)
SEASON 2015 WEEK 3 [28, 44]
KC (28) at GB (38)
SEASON 2015 WEEK 4 [20, 20]
BAL (23) a

SEASON 2015 WEEK 14 [7, 28]
DAL (7) at GB (28)
SEASON 2015 WEEK 14 [35, 6]
SEA (35) at BAL (6)
SEASON 2015 WEEK 14 [31, 24]
NYG (31) at MIA (24)
SEASON 2015 WEEK 15 [25, 31]
TB (23) at STL (31)
SEASON 2015 WEEK 15 [22, 16]
NYJ (19) at DAL (16)
SEASON 2015 WEEK 15 [35, 35]
CAR (38) at NYG (35)
SEASON 2015 WEEK 15 [39, 17]
ARI (40) at PHI (17)
SEASON 2015 WEEK 15 [17, 38]
CHI (17) at MIN (38)
SEASON 2015 WEEK 15 [23, 17]
ATL (23) at JAC (17)
SEASON 2015 WEEK 15 [16, 10]
HOU (16) at IND (10)
SEASON 2015 WEEK 15 [16, 32]
TEN (16) at NE (33)
SEASON 2015 WEEK 15 [32, 14]
KC (34) at BAL (14)
SEASON 2015 WEEK 15 [25, 35]
BUF (25) at WAS (35)
SEASON 2015 WEEK 15 [13, 30]
CLE (13) at SEA (30)
SEASON 2015 WEEK 15 [35, 20]
GB (30) at OAK (20)
SEASON 2015 WEEK 15 [27, 40]
DEN (27) at PIT (34)
SEASON 2015 WEEK 15 [14, 30]
MIA (14) at SD (30)
SEASON 2015 WEEK 15 [24, 14]
CIN (24) at SF (14)
SEASON 2015 WEEK 15 [35, 33]
DET (35) at NO (27)
SEASON 2015 WEEK 16 [23, 23]
SD (20) at OAK (23)
SEASON 2015 W

SEASON 2016 WEEK 9 [14, 19]
PIT (14) at BAL (21)
SEASON 2016 WEEK 9 [35, 10]
DAL (35) at CLE (10)
SEASON 2016 WEEK 9 [14, 19]
JAX (14) at KC (19)
SEASON 2016 WEEK 9 [35, 24]
NYJ (23) at MIA (27)
SEASON 2016 WEEK 9 [16, 16]
DET (22) at MIN (16)
SEASON 2016 WEEK 9 [23, 28]
PHI (23) at NYG (28)
SEASON 2016 WEEK 9 [13, 10]
CAR (13) at LA (10)
SEASON 2016 WEEK 9 [41, 23]
NO (41) at SF (23)
SEASON 2016 WEEK 9 [25, 32]
IND (31) at GB (26)
SEASON 2016 WEEK 9 [35, 41]
TEN (35) at SD (43)
SEASON 2016 WEEK 9 [25, 31]
BUF (25) at SEA (31)
SEASON 2016 WEEK 10 [7, 28]
CLE (7) at BAL (28)
SEASON 2016 WEEK 10 [15, 17]
KC (20) at CAR (17)
SEASON 2016 WEEK 10 [23, 21]
HOU (24) at JAX (21)
SEASON 2016 WEEK 10 [23, 23]
DEN (25) at NO (23)
SEASON 2016 WEEK 10 [9, 6]
LA (9) at NYJ (6)
SEASON 2016 WEEK 10 [15, 24]
ATL (15) at PHI (24)
SEASON 2016 WEEK 10 [10, 35]
CHI (10) at TB (36)
SEASON 2016 WEEK 10 [25, 47]
GB (25) at TEN (47)
SEASON 2016 WEEK 10 [20, 26]
MIN (20) at WAS (26)
SEASON 2016 WEEK 10 [30, 24]

AttributeError: 'NoneType' object has no attribute 'decode'